In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
# ✅ 2. import
import os
import json
import pandas as pd

# ✅ 3. 루트 경로 설정
TRAIN_ROOT = "/content/drive/MyDrive/childrenvoice/traindata"
VAL_ROOT   = "/content/drive/MyDrive/childrenvoice/valdata"
TEST_ROOT  = "/content/drive/MyDrive/childrenvoice/testdata"

# ✅ 4. split별 화자 정보
train_speakers = {
    4:  "1109",
    5:  "2299",
    6:  "2080",
    7:  "2196",
    8:  "1088",
    9:  "1025",
    10: "2336",
    11: "2370",
    12: "1058",
}

val_speakers = {
    4:  "0055",
    5:  "0002",
    6:  "0096",
    7:  "0901",
    8:  "0219",
    9:  "0018",
    10: "0009",
    11: "0422",
    12: "0034",
}

test_speakers = {
    4:  "1079",
    5:  "4858",
    6:  "5206",
    7:  "6124",
    8:  "7933",
    9:  "7894",
    10: "6152",
    11: "8573",
    12: "8838",
}


def sec_to_hms(sec):
    h = int(sec // 3600)
    m = int((sec % 3600) // 60)
    s = int(sec % 60)
    return f"{h:02d}:{m:02d}:{s:02d}"


def analyze_split(split_name, root_dir, age_speaker_dict):
    """
    split_name: "Train" / "Val" / "Test"
    root_dir:   해당 split의 루트 경로
    age_speaker_dict: {나이(int): speaker_id(str)}
    """
    split_total = 0.0
    rows = []

    print(f"\n================ {split_name} 분석 시작 =================")

    for age, speaker_id in sorted(age_speaker_dict.items()):
        speaker_dir = os.path.join(root_dir, f"age{age}", "labelling", speaker_id)
        if not os.path.isdir(speaker_dir):
            print(f"⚠️ {split_name} age{age} 폴더 없음: {speaker_dir}")
            continue

        max_len = 0.0
        min_len = float("inf")
        total_len = 0.0
        file_count = 0

        for fname in os.listdir(speaker_dir):
            if not fname.endswith(".json"):
                continue

            json_path = os.path.join(speaker_dir, fname)
            try:
                with open(json_path, "r", encoding="utf-8") as f:
                    data = json.load(f)

                # ✅ AI-Hub 구조 그대로 사용
                file_length = float(data["File"]["FileLength"])  # 초 단위
            except Exception as e:
                print(f"❌ 오류: {json_path} / {e}")
                continue

            file_count += 1
            total_len += file_length

            if file_length > max_len:
                max_len = file_length
            if file_length < min_len:
                min_len = file_length

        if file_count == 0:
            min_len = 0.0  # 데이터 없을 때 NaN 방지

        split_total += total_len

        print(
            f"{split_name} age{age} (spk {speaker_id}) "
            f"- 파일 {file_count}개, 총 {sec_to_hms(total_len)} ({total_len:.1f} sec)"
        )

        rows.append(
            {
                "age": age,
                "speaker_id": speaker_id,
                "json_count": file_count,
                "total_len_sec": round(total_len, 3),
                "total_len_hms": sec_to_hms(total_len),
                "min_len_sec": round(min_len, 3),
                "max_len_sec": round(max_len, 3),
            }
        )

    df = pd.DataFrame(rows).sort_values("age")
    print(f"\n🔹 {split_name} 요약 (age별):")
    print(df)

    print(
        f"\n👉 {split_name} 전체 길이: "
        f"{sec_to_hms(split_total)} ({split_total:.1f} sec)\n"
    )

    return df, split_total


# ✅ 5. Train / Val / Test 각각 분석
train_df, train_total = analyze_split("Train", TRAIN_ROOT, train_speakers)
val_df,   val_total   = analyze_split("Val",   VAL_ROOT,   val_speakers)
test_df,  test_total  = analyze_split("Test",  TEST_ROOT,  test_speakers)

# ✅ 6. 전체 합계
print("============== 전체 합계 ==============")
print("Train:", sec_to_hms(train_total), f"({train_total:.1f} sec)")
print("Val:  ", sec_to_hms(val_total),   f"({val_total:.1f} sec)")
print("Test: ", sec_to_hms(test_total),  f"({test_total:.1f} sec)")
print("---------------------------------------")
print(
    "Total:",
    sec_to_hms(train_total + val_total + test_total),
    f"({train_total + val_total + test_total:.1f} sec)",
)


================ Train 분석 시작 =================
Train age4 (spk 1109) - 파일 1000개, 총 01:11:37 (4297.6 sec)
Train age5 (spk 2299) - 파일 1000개, 총 01:24:39 (5079.0 sec)
Train age6 (spk 2080) - 파일 1000개, 총 01:14:04 (4444.2 sec)
Train age7 (spk 2196) - 파일 1000개, 총 02:10:38 (7838.7 sec)
Train age8 (spk 1088) - 파일 1000개, 총 01:45:50 (6350.2 sec)
Train age9 (spk 1025) - 파일 1000개, 총 01:30:58 (5458.6 sec)
Train age10 (spk 2336) - 파일 1000개, 총 02:02:13 (7333.1 sec)
Train age11 (spk 2370) - 파일 1000개, 총 02:10:13 (7813.4 sec)
Train age12 (spk 1058) - 파일 1000개, 총 01:53:08 (6788.7 sec)

🔹 Train 요약 (age별):
   age speaker_id  json_count  total_len_sec total_len_hms  min_len_sec  \
0    4       1109        1000       4297.600      01:11:37        2.048   
1    5       2299        1000       5079.040      01:24:39        2.048   
2    6       2080        1000       4444.224      01:14:04        2.368   
3    7       2196        1000       7838.720      02:10:38        3.072   
4    8       1088        1000   